In [ ]:
!pip install requests beautifulsoup4 lxml pandas tqdm

In [ ]:
!pip install geopy tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import time
import re
from geopy.geocoders import Nominatim

In [ ]:
#this function parses 1 page of krisha.kz website
def parse_krisha_page(page: int):
    url = f"https://krisha.kz/prodazha/kvartiry/almaty/?page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
                      " AppleWebKit/537.36 (KHTML, like Gecko)"
                      " Chrome/114.0.0.0 Safari/537.36"
    }

    r = requests.get(url, headers=headers, timeout=10)
    if r.status_code != 200:
        print(f"Ошибка {r.status_code} при загрузке страницы {page}")
        return []

    soup = BeautifulSoup(r.text, "lxml")
    cards = soup.find_all("div", class_="a-card__inc")
    results = []

    for card in cards:
        try:

            title_tag = card.find("a", class_="a-card__title")
            title = title_tag.text.strip() if title_tag else None
            link = (
                "https://krisha.kz" + title_tag.get("href")
                if title_tag and title_tag.get("href") else None
            )


            price_tag = card.find("div", class_="a-card__price")
            price_raw = price_tag.text.strip() if price_tag else None
            price_clean = re.sub(r"[^\d]", "", price_raw or "")


            address_tag = card.find("div", class_="a-card__subtitle")
            address = address_tag.text.strip() if address_tag else None


            rooms = None
            area = None
            if title:
                room_match = re.search(r"(\d+)[- ]*ком", title)
                area_match = re.search(r"(\d+(?:[\.,]\d+)?)\s*м²", title)
                if room_match:
                    rooms = int(room_match.group(1))
                if area_match:
                    area = float(area_match.group(1).replace(",", "."))

            results.append({
                "title": title,
                "price_raw": price_raw,
                "price": int(price_clean) if price_clean else None,
                "rooms": rooms,
                "area_m2": area,
                "address": address,
                "link": link,
            })
        except Exception as e:
            print(f"Ошибка при парсинге объявления: {e}")

    return results



In [ ]:
all_data = []
for page in tqdm(range(1, 6)):
    all_data.extend(parse_krisha_page(page))
    time.sleep(2)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(all_data)
df.drop_duplicates(subset="link", inplace=True)
df["price_per_m2"] = df.apply(
    lambda x: round(x["price"] / x["area_m2"], 2) if x["price"] and x["area_m2"] else None, axis=1
)
df.loc[df["area_m2"] < 10, "area_m2"] = None


In [ ]:
print(df.columns)

Index(['title', 'price_raw', 'price', 'rooms', 'area_m2', 'address', 'link',
       'price_per_m2'],
      dtype='object')


In [ ]:
def extract_district(address):
    if not isinstance(address, str):
        return None
    match = re.search(r"([А-Яа-яЁёA-Za-z\-]+)\s*р-н", address)
    if match:
        return match.group(1)
    return None

In [ ]:
df["district"] = df["address"].apply(extract_district)

In [ ]:
GOOGLE_API_KEY = "your_key"

In [ ]:
def geocode_google(address):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": GOOGLE_API_KEY,
        "language": "ru"
    }
    response = requests.get(url, params=params, timeout=10)
    data = response.json()
    if data.get("status") == "OK":
        loc = data["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    else:
        return None, None

In [ ]:
df = df.copy()
df["latitude"] = None
df["longitude"] = None

In [ ]:
for i, row in tqdm(df.head(30).iterrows(), total=30):
    address = row["address"]
    lat, lng = geocode_google(address)
    df.loc[i, "latitude"] = lat
    df.loc[i, "longitude"] = lng
    time.sleep(1.0)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
df = df[["title", "rooms", "area_m2", "price", "price_per_m2", "address", "district", "latitude", "longitude", "link"]]
df.to_csv("krisha_almaty_clean.csv", index=False)

In [ ]:
print(f"Успешно сохранено {len(df)} строк в 'krisha_almaty_clean.csv'")
df.head(10)

Успешно сохранено 101 строк в 'krisha_almaty_clean.csv'


,title,rooms,area_m2,price,price_per_m2,address,district,latitude,longitude,link
0,5-комнатная квартира · 217.9 м²,5,217.90,334476500,1535000.00,"Медеуский р-н, Талгат Бигелдинов 1/2",Медеуский,43.264528,76.95257,https://krisha.kz/a/show/1005842286
1,2-комнатная квартира · 62.6 м²,2,62.60,51269400,819000.00,"Жетысуский р-н, Райымбека 219Б",Жетысуский,43.268782,76.920686,https://krisha.kz/a/show/761881225
2,4-комнатная квартира · 117 м²,4,117.00,115759800,989400.00,"Ауэзовский р-н, мкр Таугуль-3, Шаймерденова 32",Ауэзовский,43.197641,76.855285,https://krisha.kz/a/show/676367958
3,1-комнатная квартира · 34 м² · 2/5 этаж,1,34.00,39500000,1161764.71,"Медеуский р-н, Жибек жолы 33 — Абдуллиных",Медеуский,43.263272,76.960119,https://krisha.kz/a/show/1005956202
4,2-комнатная квартира · 68.92 м²,2,68.92,30669400,445000.00,"Турксибский р-н, мкр Нуршашкан (Колхозши), Ала...",Турксибский,43.331758,77.020957,https://krisha.kz/a/show/1000460175
5,1-комнатная квартира · 45 м²,1,45.00,24930000,554000.00,"Турксибский р-н, Сарыарка 1/1",Турксибский,43.346737,76.968903,https://krisha.kz/a/show/687262524
6,3-комнатная квартира · 61 м² · 1/13 этаж,3,61.00,45000000,737704.92,"Алмалинский р-н, Муратбаева 14 — Муратбаева Ма...",Алмалинский,43.264855,76.918052,https://krisha.kz/a/show/1006041423
7,2-комнатная квартира · 48.84 м²,2,48.84,38583600,790000.00,"Алатауский р-н, мкр Дарабоз, Бауыржана Момышул...",Алатауский,43.275134,76.850457,https://krisha.kz/a/show/762071085
8,3-комнатная квартира · 108.27 м²,3,108.27,129924000,1200000.00,"Алмалинский р-н, Шевченко 109",Алмалинский,43.245048,76.924779,https://krisha.kz/a/show/1000752773
9,2-комнатная квартира · 45 м² · 7/8 этаж,2,45.00,40000000,888888.89,"Бостандыкский р-н, Арайлы 12 — РЕМИЗОВКА",Бостандыкский,43.189175,76.930946,https://krisha.kz/a/show/1003297275
